In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import itertools 
import matplotlib.pyplot as plt
from terminaltables import AsciiTable
from scipy.stats import chi2

C:\Users\Deutsch\AppData\Local\Temp\ipykernel_7584\3588027038.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#SE IMPORTA EL CSV
toyota_data = pd.read_csv('../CSV/Toyota2.csv')
toyota_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-04-01,118.000000,119.330002,116.500000,116.709999,116.709999,398100
1,2020-04-02,115.910004,118.120003,115.720001,117.820000,117.820000,292600
2,2020-04-03,114.809998,115.690002,113.980003,115.000000,115.000000,337100
3,2020-04-06,120.000000,122.809998,119.400002,122.410004,122.410004,328200
4,2020-04-07,125.570000,125.669998,121.900002,121.900002,121.900002,239600
...,...,...,...,...,...,...,...
980,2024-02-23,235.000000,235.759995,234.339996,235.000000,235.000000,279000
981,2024-02-26,237.589996,238.779999,236.539993,238.130005,238.130005,332900
982,2024-02-27,238.190002,238.839996,236.589996,238.039993,238.039993,301000
983,2024-02-28,237.929993,238.190002,237.000000,237.289993,237.289993,168700


In [3]:
#SE BORRAN LAS COLUMNAS QUE NO SE USARAN
toyota_data = toyota_data.drop(columns=['Open', 'High', 'Low', 'Close','Volume'])
toyota_data

,Date,Adj Close
0,2020-04-01,116.709999
1,2020-04-02,117.820000
2,2020-04-03,115.000000
3,2020-04-06,122.410004
4,2020-04-07,121.900002
...,...,...
980,2024-02-23,235.000000
981,2024-02-26,238.130005
982,2024-02-27,238.039993
983,2024-02-28,237.289993


In [4]:
#SE CALCULA EL CAMBIO PORCENTUAL DE CADA REGISTRO DE ACUERDO AL REGISTRO ANTERIOR
percentage = [0,]
for i in range(1, len(toyota_data)):
    percent = (toyota_data['Adj Close'][i] - toyota_data['Adj Close'][i-1])/toyota_data['Adj Close'][i-1]
    percentage.append(percent)
toyota_data.insert(2, 'Change %', percentage)

#SE CALCULA LA DESVIACION ESTANDAR DE LOS 20 DATOS ANTERIORES
std_dev = [0,]*20
for i in range(21, len(percentage) + 1):
    standar_dev = np.std(percentage[i-20:i])
    std_dev.append(standar_dev)
toyota_data.insert(3, 'std dev', std_dev)

#SE ASIGNA EL ESTADO DE ACUERDO A LA DESVIACION ESTANDAR DE LAS 20 SEMANAS ANTERIORES
states = [0,]*20
for j in range(20, len(percentage)):
    change = percentage[j]
    desv = std_dev[j]
    if change <= -2*desv:
        state = "B3"
    elif -2*desv < change <= -desv:
        state = "B2"
    elif -desv < change < 0:
        state = "B1"
    elif 0 <= change < desv:
        state = "S1"
    elif desv <= change < 2*desv:
        state = "S2"
    elif 2*desv <= change:  
        state = "S3"
    states.append(state)
toyota_data.insert(4, 'Current State', states)

#SE ASIGNA UNA NUEVA COLUMNA CON EL ESTADO SIGUIENTE AL REGISTRO
next_state_column=[]
for i in range(len(toyota_data)-1):
    next_state_column.append(toyota_data['Current State'][i+1])

previows_state_column=[0]
for i in range(1,len(toyota_data)-1):
    previows_state_column.append(toyota_data['Current State'][i-1])
    
#SE BORRA LA ULTIMA FILA POR QUE NO TIENE ESTADO SIGUIENTE
toyota_data = toyota_data.drop(toyota_data.index[-1])
toyota_data.insert(5, 'Next State', next_state_column)
toyota_data.insert(4, 'Previous State', previows_state_column)

complete_table = AsciiTable([toyota_data.columns]+list(toyota_data.values))
print("Complete Table \n",complete_table.table)

Complete Table 
 +------------+------------+-------------------------+-----------------------+----------------+---------------+------------+
| Date       | Adj Close  | Change %                | std dev               | Previous State | Current State | Next State |
+------------+------------+-------------------------+-----------------------+----------------+---------------+------------+
| 2020-04-01 | 116.709999 | 0.0                     | 0.0                   | 0              | 0             | 0          |
| 2020-04-02 | 117.82     | 0.009510761798567035    | 0.0                   | 0              | 0             | 0          |
| 2020-04-03 | 115.0      | -0.02393481582074345    | 0.0                   | 0              | 0             | 0          |
| 2020-04-06 | 122.410004 | 0.06443481739130436     | 0.0                   | 0              | 0             | 0          |
| 2020-04-07 | 121.900002 | -0.00416634248292321    | 0.0                   | 0              | 0             | 0   

In [5]:
def calculo_inverso(data):
    def desviacion_estandar(data):
        std_dev = []
        for i in range(len(data)):
            if i < 20:
                std_dev.append(0)
            else:
                standar_dev = np.std(data['Change %'][i-20:i+1])
                std_dev.append(standar_dev)
        return std_dev
    
    def calculo_cambios_inversos(data):
        cambio_inverso = []
        precios_estimados = []
        for i in range(len(data)):
            estado_actual = data['Current State'][i]
            precio_anterior = data['Adj Close'][i]
            cambio_previo = data['Change %'][i]
            std_dev = data['std dev'][i]
            cambio = 0
            if estado_actual == "B3":
                cambio = -2 * std_dev
            elif estado_actual == "B2":
                cambio = (-std_dev + cambio_previo) / 2
            elif estado_actual == "B1":
                cambio = cambio_previo / 2
            elif estado_actual == "S1":
                cambio = cambio_previo / 2
            elif estado_actual == "S2":
                cambio = (std_dev + cambio_previo) / 2
            elif estado_actual == "S3":
                cambio = 2 * std_dev
            cambio_inverso.append(cambio)
            precio_estimado = precio_anterior * (1 + cambio)
            precios_estimados.append(precio_estimado)
        return cambio_inverso, precios_estimados
    
    data['std dev'] = desviacion_estandar(data)
    cambio_inverso, precios_estimados = calculo_cambios_inversos(data)

    data['Inverse Change %'] = cambio_inverso
    data['Estimated Price'] = precios_estimados

    return data

toyota_data_inverse = calculo_inverso(toyota_data)

complete_table_inverse = AsciiTable([toyota_data_inverse.columns]+list(toyota_data_inverse.values))
print("Complete Table \n",complete_table_inverse.table)

Complete Table 
 +------------+------------+-------------------------+-----------------------+----------------+---------------+------------+-------------------------+--------------------+
| Date       | Adj Close  | Change %                | std dev               | Previous State | Current State | Next State | Inverse Change %        | Estimated Price    |
+------------+------------+-------------------------+-----------------------+----------------+---------------+------------+-------------------------+--------------------+
| 2020-04-01 | 116.709999 | 0.0                     | 0.0                   | 0              | 0             | 0          | 0.0                     | 116.709999         |
| 2020-04-02 | 117.82     | 0.009510761798567035    | 0.0                   | 0              | 0             | 0          | 0.0                     | 117.82             |
| 2020-04-03 | 115.0      | -0.02393481582074345    | 0.0                   | 0              | 0             | 0          | 0.0 

In [6]:
#SE QUITAN LAS PRIMERAS 20 FILAS, PUES ESTAS NO CUENTAN CON ESTADO ACTUAL
toyota_trim=toyota_data.iloc[21:]
table_week_21 = AsciiTable([toyota_trim.columns]+list(toyota_trim.values))
print("Table since week 21 \n",table_week_21.table)

Table since week 21 
 +------------+------------+-------------------------+-----------------------+----------------+---------------+------------+-------------------------+--------------------+
| Date       | Adj Close  | Change %                | std dev               | Previous State | Current State | Next State | Inverse Change %        | Estimated Price    |
+------------+------------+-------------------------+-----------------------+----------------+---------------+------------+-------------------------+--------------------+
| 2020-05-01 | 121.120003 | -0.019985379722805414   | 0.021453830209722042  | B2             | B1            | B1         | -0.009992689861402707   | 119.90968837400884 |
| 2020-05-04 | 120.699997 | -0.003467684854664351   | 0.021415316824651264  | B1             | B1            | S1         | -0.0017338424273321755  | 120.49072222422252 |
| 2020-05-05 | 121.959999 | 0.010439122049025404    | 0.020720493004081447  | B1             | S1            | B1         |

In [7]:
#FUNCIONES PARA SOLUCIONAR DIVISIONES POR 0
def resizeExpectedValues(freq_matrix, states):
    index_to_delete = []
    for index in range(len(freq_matrix)):
        if 0 in freq_matrix[index]:
            if index > 0:
                freq_matrix[index-1] += freq_matrix[index]
                freq_matrix[:,index-1] += freq_matrix[:,index] 
            else:
                freq_matrix[index+1] += freq_matrix[index]
                freq_matrix[:,index+1] += freq_matrix[:,index] 
            index_to_delete.append(index)
    freq_matrix = np.delete(freq_matrix, index_to_delete, axis=0)
    freq_matrix = np.delete(freq_matrix, index_to_delete, axis=1)
    
    for i in range(len(states)):
        if i in index_to_delete:
            if index > 0:
                states[i-1]=f"{states[i-1]}+{states[i]}"
            else:
                states[i+1]=f"{states[i]}+{states[i+1]}"
    states=np.delete(states,index_to_delete)
    return freq_matrix, states

In [8]:
def validate_markov_rule(data):

    #PROBABILIDAD ESPERADA
    states=np.unique(data['Current State'])
    states_mapping = {}
    for i,state in enumerate(states):
        states_mapping[state]=i

    #SE CREA LA MATRIZ DE FRECUENCIAS ESPERADAS
    freq_expected=np.zeros(((len(states_mapping)),len(states_mapping)))
    for i in range(len(data)):
        freq_expected[states_mapping[data.iloc[i]['Current State']], states_mapping[data.iloc[i]['Next State']]] += 1
    #FUNCION PARA FRECUENCIAS DE 0
    if 0 in freq_expected:
        freq_expected, states=resizeExpectedValues(freq_expected,states)
        states_mapping={new_state: i for i,new_state in enumerate(states)}
        
    #SE DETERMINA LA MATRIZ DE PROBABILIDAD ESPERADA
    transition_expected = np.zeros(((len(states)), len(states)))
    for i in range(len(freq_expected)):
        transition_expected[i] = freq_expected[i] / sum(freq_expected[i])

    data_frame_expected = pd.DataFrame(transition_expected, columns=states, index=states)
    print(f"Probabilidad Esperada\n{data_frame_expected}\n")
    transition_expected=np.tile(transition_expected,(len(transition_expected),1))
    
    #PROBABILIDAD OBSERVADA
    dict_states_before = {prev_state :{curr_state: {next_state: 0 for next_state in states} for curr_state in states} for prev_state in states}
    states_before = list(itertools.product(states,repeat=2))
    states_list_before = []
    for i,state_before in enumerate(states_before):
        complete_state=f"{state_before[0]} - {state_before[1]}"
        states_list_before.append(complete_state)
    
    #SE CREA LA MATRIZ DE FRECUENCIAS OBSERVADA
    for i in range(len(data)):
        row_in_data=data.iloc[i]
        prev_state = row_in_data['Previous State']
        curr_state = row_in_data['Current State']
        next_state = row_in_data['Next State']
        for _,prev_state_map in enumerate(states):
            if prev_state in prev_state_map:
                for _,curr_state_map in enumerate(states):
                    if curr_state in curr_state_map:
                        for _,next_state_map in enumerate(states):
                            if next_state in next_state_map:
                                dict_states_before[prev_state_map][curr_state_map][next_state_map] += 1

    freq_observed=np.zeros(((len(states_list_before)),len(states)))
    for i,state_i in enumerate(states):
        for j,state_j in enumerate(states):
            for z, state_z, in enumerate(states):
                coordinate_y=j+i*len(dict_states_before)
                freq_observed[coordinate_y,z]=dict_states_before[state_i][state_j][state_z]

    #SE DETERMINA LA MATRIZ DE PROBABILIDAD OBSERVADA
    transitions_observed=np.zeros(((len(freq_observed)),len(states)))
    for i in range(len(freq_observed)):
        transitions_observed[i]=freq_observed[i]/sum(freq_observed[i])

    data_frame_observed=pd.DataFrame(transitions_observed,columns=states,index=states_list_before)
    print(f"Probabilidad Observada \n {data_frame_observed}\n")

    #CHI CUADRADO
    chi2_observed = (np.square(transitions_observed - transition_expected)) / transition_expected
    for i in range(len(chi2_observed)):
        chi2_observed[i]=chi2_observed[i]*sum(freq_observed[i])
    chi2_observed=np.sum(chi2_observed)

    #VALORES NECESARIOS
    rows,columns=transitions_observed.shape
    grades=(rows-1)*(columns-1)
    alpha=0.05
    chi2_theory=chi2.ppf(1-alpha,grades)
    accepted=chi2_observed<chi2_theory

    print(f"Número de Filas: {rows}")
    print(f"Número de Columnas: {columns}")
    print(f"Grados de Libertad: {grades}")
    print(f"Porcentaje de Confianza: {(1-alpha)*100}%")
    print(f"Chi Cuadrado: {chi2_observed}")
    print(f"Chi Teórico: {chi2_theory}")
    print(f"¿Se Acepta?: {accepted}")
    return accepted

is_accepted=validate_markov_rule(toyota_trim[['Previous State', 'Current State', 'Next State']])

Probabilidad Esperada
          B1        B2        B3        S1        S2        S3
B1  0.334405  0.106109  0.032154  0.405145  0.109325  0.012862
B2  0.227273  0.136364  0.045455  0.372727  0.136364  0.081818
B3  0.275862  0.103448  0.034483  0.344828  0.206897  0.034483
S1  0.357746  0.115493  0.025352  0.343662  0.132394  0.025352
S2  0.304000  0.120000  0.024000  0.352000  0.144000  0.056000
S3  0.242424  0.090909  0.030303  0.393939  0.151515  0.090909

Probabilidad Observada 
                B1        B2        B3        S1        S2        S3
B1 - B1  0.298077  0.115385  0.038462  0.432692  0.105769  0.009615
B1 - B2  0.242424  0.151515  0.030303  0.333333  0.151515  0.090909
B1 - B3  0.100000  0.000000  0.000000  0.700000  0.200000  0.000000
B1 - S1  0.384000  0.088000  0.024000  0.376000  0.120000  0.008000
B1 - S2  0.352941  0.088235  0.000000  0.382353  0.088235  0.088235
B1 - S3  0.000000  0.000000  0.250000  0.500000  0.250000  0.000000
B2 - B1  0.500000  0.038462  0.1153

In [9]:
def markov_model(data):
    #PROBABILIDAD OBSERVADA
    states=np.unique(data['Current State'])
    states_mapping = {}
    for i,state in enumerate(states):
        states_mapping[state]=i

    #PROBABILIDAD ESPERADA
    transition_matrix = np.zeros(((len(states_mapping)), len(states_mapping)))
    for i in range(len(data)):
        transition_matrix[states_mapping[data.iloc[i]['Current State']], states_mapping[data.iloc[i]['Next State']]] += 1
    for i in range(len(transition_matrix)):
        transition_matrix[i] = transition_matrix[i] / sum(transition_matrix[i])
    data_frame_expected = pd.DataFrame(transition_matrix, columns=list(states_mapping.keys()), index=list(states_mapping.keys()))
    print(f"Probabilidad Esperada\n{data_frame_expected}\n")

    return transition_matrix
#SE ENVIAN LAS COLUMNAS DE ESTADO ACTUAL Y SIGUIENTE
transition_matrix = markov_model(toyota_trim[['Current State', 'Next State']])

Probabilidad Esperada
          B1        B2        B3        S1        S2        S3
B1  0.334405  0.106109  0.032154  0.405145  0.109325  0.012862
B2  0.227273  0.136364  0.045455  0.372727  0.136364  0.081818
B3  0.275862  0.103448  0.034483  0.344828  0.206897  0.034483
S1  0.357746  0.115493  0.025352  0.343662  0.132394  0.025352
S2  0.304000  0.120000  0.024000  0.352000  0.144000  0.056000
S3  0.242424  0.090909  0.030303  0.393939  0.151515  0.090909



In [10]:
def sendInfoToAnalize():
    return is_accepted, transition_matrix